In [3]:
from pathlib import Path
import pandas as pd

# Use current directory
data_dir = Path(".")

# Look for all STD CSVs
std_files = list(data_dir.glob("*__STD_*.csv"))

# Debug: list all matched files
print("Found STD files:", [f.name for f in std_files])

all_dfs = []

for file in std_files:
    try:
        df = pd.read_csv(file)

        print(f"\n[OK] Reading file: {file.name}")
        print(f"[INFO] Columns: {df.columns.tolist()}")

        bank_name = file.stem.split("__STD_")[-1]
        df["Bank_Name"] = bank_name

        standard_columns = [
            "Transaction_Date", "Description", "Debit_Amount",
            "Credit_Amount", "Balance", "Bank_Name"
        ]

        # Add missing columns
        for col in standard_columns:
            if col not in df.columns:
                print(f"[WARNING] Missing column: {col} -> Adding blank column")
                df[col] = ""

        # Reorder
        df = df[standard_columns]

        # Normalize values
        for amt_col in ["Debit_Amount", "Credit_Amount", "Balance"]:
            df[amt_col] = pd.to_numeric(df[amt_col], errors='coerce').fillna(0.0)

        # Convert date
        df["Transaction_Date"] = pd.to_datetime(df["Transaction_Date"], errors='coerce')
        df["Transaction_Date"] = df["Transaction_Date"].dt.strftime("%Y-%m-%d")

        all_dfs.append(df)

    except Exception as e:
        print(f"[ERROR] {file.name}: {e}")

# Final CSV Output
if all_dfs:
    final_df = pd.concat(all_dfs, ignore_index=True)
    final_df.to_csv("transactions_dataset.csv", index=False)
    print("Combined dataset saved as: transactions_dataset.csv")
    display(final_df.head())
else:
    print("WARNING: No valid dataframes to combine.")


📁 Found STD files: ['3e7a0a30_Acct_Statement_XX5782_17082025__STD_HDFC.csv', 'abaa7fd0_AST1755014126988__STD_KOTAK.csv', 'ac929ae0_cub_bank_statememnt__STD_CUB.csv']

[✅] Reading file: 3e7a0a30_Acct_Statement_XX5782_17082025__STD_HDFC.csv
[🔍] Columns: ['Transaction_ID', 'Transaction_Date', 'Description', 'Debit_Amount', 'Credit_Amount', 'Balance', 'Bank_Name']

[✅] Reading file: abaa7fd0_AST1755014126988__STD_KOTAK.csv
[🔍] Columns: ['Transaction_ID', 'Transaction_Date', 'Description', 'Debit_Amount', 'Credit_Amount', 'Balance', 'Bank_Name']

[✅] Reading file: ac929ae0_cub_bank_statememnt__STD_CUB.csv
[🔍] Columns: ['Transaction_ID', 'Transaction_Date', 'Description', 'Debit_Amount', 'Credit_Amount', 'Balance', 'Bank_Name']
✅ Combined dataset saved as: transactions_dataset.csv


,Transaction_Date,Description,Debit_Amount,Credit_Amount,Balance,Bank_Name
0,2025-02-02,UPI-ZOMATO 0000503337661345 LTD-ZOMATO-ORDER@P...,306.25,0.0,101506.06,HDFC
1,2025-02-02,UPI-MURUGADOSS 0000503330329806 M-9444387308@O...,300.00,0.0,101206.06,HDFC
2,2025-02-02,UPI-RAZORPAYREDBUS-REDBUS-PAYMENT@ICICI- 00005...,1715.95,0.0,99490.11,HDFC
3,2025-02-02,UPI-MEENAKSHI 0000503335966668 S-MEENAKSHISANK...,10000.00,0.0,89490.11,HDFC
4,2025-03-02,UPI-RIZWAN-GPAYRIZWAN546@OKICICI-KKBK000 00005...,76.00,0.0,89414.11,HDFC


In [4]:
import pandas as pd
from pathlib import Path
import re
from datetime import datetime

# ---------- UTILITIES ----------
def to_float(value):
    try:
        return float(str(value).replace(",", "").strip())
    except:
        return 0.0

def is_float(val):
    try:
        float(str(val).replace(",", "").strip())
        return True
    except:
        return False

def standardize_date(date_str):
    for fmt in ("%d/%m/%y", "%d-%m-%Y", "%d-%b-%Y"):
        try:
            return datetime.strptime(date_str.strip(), fmt).strftime("%Y-%m-%d")
        except:
            continue
    return ""

# ---------- BANK-SPECIFIC CLEANING ----------
def clean_reject_rows(df, bank):
    cleaned_rows = []
    for i, row in df.iterrows():
        values = row.dropna().values
        if len(values) < 4:
            continue  # skip empty

        try:
            date = standardize_date(values[0])
            desc = values[1]

            debit, credit, balance = 0.0, 0.0, 0.0

            if bank == "HDFC":
                if len(values) >= 6:
                    debit = to_float(values[4])
                    credit = to_float(values[5])
                    balance = to_float(values[6]) if len(values) > 6 else 0.0
            elif bank == "KOTAK":
                # Kotak: values[4] contains "(Dr)" or "(Cr)"
                amount_field = str(values[4])
                if "(Dr)" in amount_field:
                    debit = to_float(amount_field.replace("(Dr)", ""))
                elif "(Cr)" in amount_field:
                    credit = to_float(amount_field.replace("(Cr)", ""))
                balance = to_float(values[5]) if len(values) > 5 else 0.0
            elif bank == "CUB":
                # Try guessing based on field length
                for val in values:
                    if is_float(val):
                        if debit == 0.0:
                            debit = to_float(val)
                        elif credit == 0.0:
                            credit = to_float(val)
                        else:
                            balance = to_float(val)

            cleaned_rows.append({
                "Transaction_Date": date,
                "Description": str(desc).strip(),
                "Debit_Amount": debit,
                "Credit_Amount": credit,
                "Balance": balance,
                "Bank_Name": bank.upper()
            })

        except Exception as e:
            print(f"ERROR: Skipped row {i}: {e}")

    return cleaned_rows

# ---------- MAIN WORKFLOW ----------
def clean_all_reject_files():
    reject_files = list(Path(".").glob("*__REJECTS_*.csv"))
    if not reject_files:
        print("ERROR: No reject files found.")
        return

    for file in reject_files:
        bank = re.search(r"__REJECTS_(\w+)\.csv", file.name)
        if not bank:
            print(f"ERROR: Could not detect bank from {file.name}")
            continue

        bank_name = bank.group(1).upper()
        print(f"\nProcessing: {file.name} ({bank_name})")

        try:
            df = pd.read_csv(file)
            cleaned = clean_reject_rows(df, bank_name)

            if cleaned:
                result_df = pd.DataFrame(cleaned)
                result_df.insert(0, "Transaction_ID", [f"{bank_name[:3]}_{i+1:05d}" for i in range(len(result_df))])
                output_file = file.name.replace("__REJECTS_", "__RECOVERED_")
                result_df.to_csv(output_file, index=False)
                print(f"Saved: {output_file}")
            else:
                print("WARNING: No valid rows recovered.")

        except Exception as e:
            print(f"ERROR: Error in {file.name}: {e}")

# ---------- RUN ----------
if __name__ == "__main__":
    clean_all_reject_files()


ERROR: Could not detect bank from 3e7a0a30_Acct_Statement_XX5782_17082025__REJECTS_HDFC (1).csv

Processing: 3e7a0a30_Acct_Statement_XX5782_17082025__REJECTS_HDFC.csv (HDFC)
Saved: 3e7a0a30_Acct_Statement_XX5782_17082025__RECOVERED_HDFC.csv

Processing: abaa7fd0_AST1755014126988__REJECTS_KOTAK.csv (KOTAK)
ERROR: Error in abaa7fd0_AST1755014126988__REJECTS_KOTAK.csv: No columns to parse from file

Processing: ac929ae0_cub_bank_statememnt__REJECTS_CUB.csv (CUB)
ERROR: Error in ac929ae0_cub_bank_statememnt__REJECTS_CUB.csv: No columns to parse from file


In [5]:
import pandas as pd
from pathlib import Path
import uuid

# Paths
final_csv_path = Path("transactions_dataset.csv")
recovered_hdfc_path = Path("3e7a0a30_Acct_Statement_XX5782_17082025__RECOVERED_HDFC.csv")

# Load both datasets
final_df = pd.read_csv(final_csv_path)
recovered_df = pd.read_csv(recovered_hdfc_path)

# Ensure required columns are present
required_cols = ['Transaction_Date', 'Description', 'Debit_Amount', 'Credit_Amount', 'Balance', 'Bank_Name']

missing_cols = [col for col in required_cols if col not in recovered_df.columns]
if missing_cols:
    print(f"WARNING: Missing columns in recovered HDFC: {missing_cols}")
    for col in missing_cols:
        recovered_df[col] = ""  # Fill missing columns with empty

# Assign Transaction_ID if not present
if "Transaction_ID" not in recovered_df.columns:
    recovered_df.insert(0, "Transaction_ID", [str(uuid.uuid4())[:8] for _ in range(len(recovered_df))])

# Align column order to match final_df
recovered_df = recovered_df[final_df.columns]

# Merge
merged_df = pd.concat([final_df, recovered_df], ignore_index=True)

# Save
merged_df.to_csv("transactions_dataset.csv", index=False)
print("Merged recovered HDFC data into transactions_dataset.csv")


Merged recovered HDFC data into transactions_dataset.csv


In [9]:
import pandas as pd

# Load the recovered file
input_file = "3e7a0a30_Acct_Statement_XX5782_17082025__RECOVERED_HDFC.csv"
output_file = "3e7a0a30_Acct_Statement_XX5782_17082025__FIXED_HDFC.csv"

# Read the single-row file
df = pd.read_csv(input_file)

# Extract the first row and convert each field to list
fields = ["Transaction_Date", "Description", "Debit_Amount", "Credit_Amount", "Balance"]
columns_data = {}

for field in fields:
    value = df.at[0, field]
    lines = str(value).split("\n") if pd.notna(value) else []
    columns_data[field] = lines

# Determine max row count
max_rows = max(len(v) for v in columns_data.values())

# Pad shorter columns
for key in columns_data:
    while len(columns_data[key]) < max_rows:
        columns_data[key].append("")

# Build cleaned rows
cleaned_rows = []
for i in range(max_rows):
    cleaned_rows.append({
        "Transaction_ID": f"HDFC_{i+1:05d}",
        "Transaction_Date": columns_data["Transaction_Date"][i].strip(),
        "Description": columns_data["Description"][i].strip(),
        "Debit_Amount": columns_data["Debit_Amount"][i].strip(),
        "Credit_Amount": columns_data["Credit_Amount"][i].strip(),
        "Balance": columns_data["Balance"][i].strip(),
        "Bank_Name": "HDFC"
    })

# Final DataFrame
clean_df = pd.DataFrame(cleaned_rows)

# Save
clean_df.to_csv(output_file, index=False)
print(f"Saved FIXED file as: {output_file}")
print("Final rows:", len(clean_df))
display(clean_df.head())


Saved FIXED file as: 3e7a0a30_Acct_Statement_XX5782_17082025__FIXED_HDFC.csv
Final rows: 1


,Transaction_ID,Transaction_Date,Description,Debit_Amount,Credit_Amount,Balance,Bank_Name
0,HDFC_00001,,31/07/25,293990.58,43071.05,0.0,HDFC
